In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
import time

### Dataset:

In [2]:
try:
    from fddbenchmark import FDDDataset, FDDDataloader, FDDEvaluator
except:
    !git clone https://github.com/airi-industrial-ai/fddbenchmark
    from fddbenchmark import FDDDataset, FDDDataloader, FDDEvaluator

In [3]:
tep_dataset = FDDDataset(name='reinartz_tep', splitting_type='supervised')
tep_dataset.df = (tep_dataset.df - tep_dataset.df.mean())/tep_dataset.df.std()
tep_dataset.df.head()

Reading data/reinartz_tep/test_mask.csv: 100%|██████████| 5600000/5600000 [00:01<00:00, 3123157.06it/s]


xmeas_1   xmeas_2   xmeas_3   xmeas_4   xmeas_5   xmeas_6  \
run_id     sample                                                               
1334009671 1      -0.175458  0.097509 -0.092930  0.022772  0.079000  0.056226   
           2      -0.158513 -0.887449 -0.243139  0.211698 -0.352012 -0.553442   
           3      -0.166950  1.864230 -0.384321 -0.192391 -0.540460 -0.496256   
           4      -0.165755 -0.425304 -0.289450  0.085419  1.186139  0.114638   
           5      -0.179773  1.085111 -0.254781  0.393148 -0.116478 -0.816772   

                    xmeas_7   xmeas_8   xmeas_9  xmeas_10  ...     xmv_2  \
run_id     sample                                          ...             
1334009671 1       0.057243  0.078469 -0.042816 -0.081580  ... -0.101671   
           2      -0.006126 -0.248531 -0.358203 -0.098747  ... -0.037165   
           3       0.047474  0.041677  0.116272 -0.115753  ... -0.131093   
           4      -0.020781  0.570426  0.066785 -0.146494  ... -0.067351   
           5       0.051283 -0.513587 -0.061174 -0.184990  ... -0.059039   

                      xmv_3     xmv_4  xmv_5     xmv_6     xmv_7     xmv_8  \
run_id     sample                                                            
1334009671 1      -0.237457 -0.139339    NaN -0.082903  0.236431  0.025629   
           2      -0.234886 -0.141768    NaN -0.105557  0.236835  0.066062   
           3      -0.232738 -0.140334    NaN -0.120353  0.108759 -0.071836   
           4      -0.231545 -0.160266    NaN -0.144987  0.036609 -0.135977   
           5      -0.230173 -0.122039    NaN -0.166149  0.122855  0.056399   

                   xmv_9    xmv_10    xmv_11  
run_id     sample                             
1334009671 1         NaN -0.149639 -1.448923  
           2         NaN -0.203725 -0.178945  
           3         NaN  0.048110  0.073242  
           4         NaN -0.178125  0.081174  
           5         NaN -0.243887  1.057207  

[5 rows x 52 columns]

In [4]:
tep_dataset.df['xmv_5'] = 1.0
tep_dataset.df['xmv_9'] = 1.0

### Dataloader:

In [5]:
train_dl = FDDDataloader(
    dataframe=tep_dataset.df,
    mask=tep_dataset.train_mask,
    labels=tep_dataset.labels,
    window_size=100,
    step_size=1,
    minibatch_training=True,
    batch_size=512,
    shuffle=True
)

test_dl = FDDDataloader(
    dataframe=tep_dataset.df,
    mask=tep_dataset.test_mask,
    labels=tep_dataset.labels,
    window_size=100, 
    step_size=1, 
    minibatch_training=True,
    batch_size=512
)

Creating sequence of samples: 100%|██████████| 560/560 [00:24<00:00, 23.09it/s]


### Model:

In [6]:
num_nodes = 52 # number of nodes
window_size = 100 # window size
ngnn = 1 # number of graph structure learning layers
gsllayer = 'directed' # type of graph structure learning layer ('relu', 'directed', 'unidirected', 'undirected')
nhidden = 256 # feature size in GCN layers
alpha = 0.1 # hyperparameter for weights of edges
k = None # max number of edges for each node
out_channels = 29 # number of process states

In [7]:
from models import *

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = GNNTEP(num_nodes, window_size, ngnn, gsllayer, nhidden,
               alpha, k, out_channels, device)

### Training:

In [8]:
model.to(device)
optimizer = Adam(model.parameters(), lr=0.001)
n_epochs = 50

In [9]:
%%time
for e in range(n_epochs):
    model.train()
    av_loss = []
    for train_ts, train_index, train_label in train_dl:
        m = torch.FloatTensor(train_ts)
        v_train_ts = torch.transpose(m, 1, 2)
        train_label = torch.LongTensor(train_label.values).to(device)
        weight = torch.ones(29) * 0.5
        weight[1:] /= 20
        logits = model(v_train_ts)
        loss = F.cross_entropy(logits, train_label, weight=weight.to(device))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        av_loss.append(loss.item())
    print(f'Epoch: {e+1:2d}/{n_epochs}, average CE loss: {sum(av_loss)/len(av_loss):.4f}')
    
    preds = []
    test_labels = []
    model.eval()
    for test_ts, test_index, test_label in test_dl:
        m = torch.FloatTensor(test_ts)
        v_test_ts = torch.transpose(m, 1, 2)
        with torch.no_grad():
            logits = model(v_test_ts)
        pred = logits.argmax(axis=1).cpu().numpy()
        preds.append(pd.Series(pred, index=test_index))
        test_labels.append(test_label)
    pred = pd.concat(preds)
    test_label = pd.concat(test_labels)
    
    evaluator = FDDEvaluator(
    splitting_type=tep_dataset.splitting_type, 
    step_size=test_dl.step_size
    )
    evaluator.print_metrics(test_label, pred)
    
    torch.save(model, 'directed_epoch_'+str(e+1)+'.pth')

Epoch:  1/50, average CE loss: 0.1390
FDD metrics
-----------------
TPR/FPR:
    Fault 01: 0.9966/0.0000
    Fault 02: 0.9926/0.0000
    Fault 03: 0.9853/0.0000
    Fault 04: 0.9982/0.0000
    Fault 05: 0.9831/0.0000
    Fault 06: 0.9993/0.0000
    Fault 07: 0.9986/0.0000
    Fault 08: 0.9706/0.0000
    Fault 09: 0.6119/0.0000
    Fault 10: 0.9721/0.0000
    Fault 11: 0.9895/0.0000
    Fault 12: 0.9781/0.0000
    Fault 13: 0.9684/0.0000
    Fault 14: 0.9957/0.0000
    Fault 15: 0.0002/0.0000
    Fault 16: 0.5694/0.0005
    Fault 17: 0.9779/0.0000
    Fault 18: 0.9586/0.0000
    Fault 19: 0.9900/0.0000
    Fault 20: 0.9690/0.0000
    Fault 21: 0.0000/0.0000
    Fault 22: 0.6753/0.0000
    Fault 23: 0.4310/0.0000
    Fault 24: 0.9768/0.0000
    Fault 25: 0.9850/0.0000
    Fault 26: 0.9806/0.0000
    Fault 27: 0.9906/0.0000
    Fault 28: 0.9339/0.0000
Detection TPR: 0.8768
Detection FPR: 0.0005
Average Detection Delay (ADD): 37.46
Total Correct Diagnosis Rate (Total CDR): 0.9727

Clusteri

Epoch:  9/50, average CE loss: 0.0710
FDD metrics
-----------------
TPR/FPR:
    Fault 01: 0.9972/0.0000
    Fault 02: 0.9943/0.0000
    Fault 03: 0.9896/0.0000
    Fault 04: 0.9986/0.0000
    Fault 05: 0.9921/0.0000
    Fault 06: 0.9992/0.0000
    Fault 07: 0.9986/0.0000
    Fault 08: 0.9812/0.0000
    Fault 09: 0.6184/0.0000
    Fault 10: 0.9786/0.0000
    Fault 11: 0.9911/0.0000
    Fault 12: 0.9860/0.0000
    Fault 13: 0.9734/0.0000
    Fault 14: 0.9964/0.0000
    Fault 15: 0.2974/0.0036
    Fault 16: 0.8065/0.0006
    Fault 17: 0.9798/0.0000
    Fault 18: 0.9634/0.0000
    Fault 19: 0.9913/0.0000
    Fault 20: 0.9701/0.0000
    Fault 21: 0.0000/0.0000
    Fault 22: 0.7389/0.0000
    Fault 23: 0.8521/0.0000
    Fault 24: 0.9901/0.0000
    Fault 25: 0.9878/0.0000
    Fault 26: 0.9822/0.0000
    Fault 27: 0.9935/0.0000
    Fault 28: 0.9578/0.0000
Detection TPR: 0.9158
Detection FPR: 0.0043
Average Detection Delay (ADD): 42.20
Total Correct Diagnosis Rate (Total CDR): 0.9751

Clusteri

Epoch: 17/50, average CE loss: 0.0654
FDD metrics
-----------------
TPR/FPR:
    Fault 01: 0.9974/0.0000
    Fault 02: 0.9940/0.0000
    Fault 03: 0.9866/0.0000
    Fault 04: 0.9986/0.0000
    Fault 05: 0.8821/0.0000
    Fault 06: 0.9993/0.0000
    Fault 07: 0.9986/0.0000
    Fault 08: 0.9804/0.0000
    Fault 09: 0.6613/0.0000
    Fault 10: 0.9760/0.0000
    Fault 11: 0.9921/0.0000
    Fault 12: 0.9878/0.0000
    Fault 13: 0.9728/0.0000
    Fault 14: 0.9964/0.0000
    Fault 15: 0.3396/0.0040
    Fault 16: 0.8608/0.0003
    Fault 17: 0.9799/0.0000
    Fault 18: 0.9647/0.0000
    Fault 19: 0.9908/0.0000
    Fault 20: 0.9705/0.0000
    Fault 21: 0.0000/0.0000
    Fault 22: 0.6946/0.0000
    Fault 23: 0.9309/0.0006
    Fault 24: 0.9898/0.0000
    Fault 25: 0.9880/0.0000
    Fault 26: 0.9824/0.0000
    Fault 27: 0.9924/0.0000
    Fault 28: 0.9599/0.0000
Detection TPR: 0.9189
Detection FPR: 0.0049
Average Detection Delay (ADD): 37.87
Total Correct Diagnosis Rate (Total CDR): 0.9743

Clusteri

Epoch: 25/50, average CE loss: 0.0624
FDD metrics
-----------------
TPR/FPR:
    Fault 01: 0.9975/0.0000
    Fault 02: 0.9933/0.0000
    Fault 03: 0.9862/0.0000
    Fault 04: 0.9985/0.0000
    Fault 05: 0.9919/0.0002
    Fault 06: 0.9993/0.0000
    Fault 07: 0.9985/0.0000
    Fault 08: 0.9800/0.0000
    Fault 09: 0.6702/0.0000
    Fault 10: 0.9766/0.0000
    Fault 11: 0.9919/0.0000
    Fault 12: 0.9875/0.0000
    Fault 13: 0.9732/0.0000
    Fault 14: 0.9965/0.0000
    Fault 15: 0.3712/0.0051
    Fault 16: 0.8855/0.0002
    Fault 17: 0.9801/0.0000
    Fault 18: 0.9648/0.0000
    Fault 19: 0.9902/0.0000
    Fault 20: 0.9702/0.0000
    Fault 21: 0.0000/0.0000
    Fault 22: 0.6825/0.0000
    Fault 23: 0.9337/0.0004
    Fault 24: 0.9900/0.0000
    Fault 25: 0.9880/0.0000
    Fault 26: 0.9820/0.0000
    Fault 27: 0.9919/0.0000
    Fault 28: 0.9611/0.0000
Detection TPR: 0.9250
Detection FPR: 0.0059
Average Detection Delay (ADD): 34.80
Total Correct Diagnosis Rate (Total CDR): 0.9743

Clusteri

Epoch: 33/50, average CE loss: 0.0605
FDD metrics
-----------------
TPR/FPR:
    Fault 01: 0.9974/0.0000
    Fault 02: 0.9928/0.0000
    Fault 03: 0.9862/0.0000
    Fault 04: 0.9985/0.0000
    Fault 05: 0.9924/0.0002
    Fault 06: 0.9993/0.0000
    Fault 07: 0.9986/0.0000
    Fault 08: 0.9797/0.0000
    Fault 09: 0.6888/0.0001
    Fault 10: 0.9763/0.0000
    Fault 11: 0.9916/0.0000
    Fault 12: 0.9869/0.0000
    Fault 13: 0.9735/0.0000
    Fault 14: 0.9966/0.0000
    Fault 15: 0.4315/0.0071
    Fault 16: 0.9085/0.0004
    Fault 17: 0.9798/0.0000
    Fault 18: 0.9649/0.0000
    Fault 19: 0.9913/0.0000
    Fault 20: 0.9704/0.0000
    Fault 21: 0.0000/0.0000
    Fault 22: 0.6627/0.0000
    Fault 23: 0.9403/0.0004
    Fault 24: 0.9909/0.0000
    Fault 25: 0.9884/0.0000
    Fault 26: 0.9825/0.0000
    Fault 27: 0.9921/0.0000
    Fault 28: 0.9581/0.0000
Detection TPR: 0.9285
Detection FPR: 0.0082
Average Detection Delay (ADD): 34.18
Total Correct Diagnosis Rate (Total CDR): 0.9739

Clusteri

Epoch: 41/50, average CE loss: 0.0590
FDD metrics
-----------------
TPR/FPR:
    Fault 01: 0.9974/0.0000
    Fault 02: 0.9919/0.0000
    Fault 03: 0.9868/0.0000
    Fault 04: 0.9985/0.0000
    Fault 05: 0.9931/0.0001
    Fault 06: 0.9993/0.0000
    Fault 07: 0.9986/0.0000
    Fault 08: 0.9813/0.0000
    Fault 09: 0.6500/0.0001
    Fault 10: 0.9768/0.0000
    Fault 11: 0.9923/0.0000
    Fault 12: 0.9871/0.0000
    Fault 13: 0.9725/0.0000
    Fault 14: 0.9966/0.0000
    Fault 15: 0.4284/0.0072
    Fault 16: 0.9126/0.0004
    Fault 17: 0.9801/0.0000
    Fault 18: 0.9644/0.0000
    Fault 19: 0.9904/0.0000
    Fault 20: 0.9701/0.0000
    Fault 21: 0.0000/0.0000
    Fault 22: 0.6931/0.0000
    Fault 23: 0.9426/0.0004
    Fault 24: 0.9908/0.0000
    Fault 25: 0.9886/0.0000
    Fault 26: 0.9830/0.0000
    Fault 27: 0.9913/0.0000
    Fault 28: 0.9590/0.0000
Detection TPR: 0.9286
Detection FPR: 0.0084
Average Detection Delay (ADD): 31.90
Total Correct Diagnosis Rate (Total CDR): 0.9736

Clusteri

Epoch: 49/50, average CE loss: 0.0580
FDD metrics
-----------------
TPR/FPR:
    Fault 01: 0.9974/0.0000
    Fault 02: 0.9925/0.0000
    Fault 03: 0.9862/0.0000
    Fault 04: 0.9985/0.0000
    Fault 05: 0.9914/0.0001
    Fault 06: 0.9993/0.0000
    Fault 07: 0.9985/0.0000
    Fault 08: 0.9780/0.0000
    Fault 09: 0.6211/0.0001
    Fault 10: 0.9767/0.0000
    Fault 11: 0.9927/0.0000
    Fault 12: 0.9874/0.0000
    Fault 13: 0.9747/0.0000
    Fault 14: 0.9965/0.0000
    Fault 15: 0.4584/0.0086
    Fault 16: 0.9113/0.0003
    Fault 17: 0.9799/0.0000
    Fault 18: 0.9645/0.0000
    Fault 19: 0.9889/0.0000
    Fault 20: 0.9702/0.0000
    Fault 21: 0.0000/0.0000
    Fault 22: 0.7109/0.0000
    Fault 23: 0.9452/0.0004
    Fault 24: 0.9903/0.0000
    Fault 25: 0.9877/0.0000
    Fault 26: 0.9826/0.0000
    Fault 27: 0.9900/0.0000
    Fault 28: 0.9583/0.0000
Detection TPR: 0.9299
Detection FPR: 0.0097
Average Detection Delay (ADD): 31.25
Total Correct Diagnosis Rate (Total CDR): 0.9728

Clusteri